In [1]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

/home/rzaharov@mvs.local/GB/venv_course/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [44]:
!ls ../2

187.zip  image		 lesson_2_v01.ipynb  lesson-2.zip  prepared_answers.txt
corpus	 lesson_2.ipynb  lesson_2_v02.ipynb  Otvety.txt


In [52]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("../2/prepared_answers.txt", "w") as fout:
    with open("../2/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-52-cbb65dc45a9b>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


In [2]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [3]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [4]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("../2/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

<ipython-input-4-3b74e1674ab8>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


In [5]:
sentences = [i for i in sentences if len(i) > 2]
print('start train fasttext')
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5, workers=8)
modelFT.save("ft_model")

start train fasttext


In [4]:
modelFT = FastText.load("ft_model")

In [6]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("../2/prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('speaker.ann')

<ipython-input-6-cf8a320e384c>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


<ipython-input-6-cf8a320e384c>:15: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
<ipython-input-6-cf8a320e384c>:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word]


True

In [7]:
modelFT = FastText.load("ft_model")

In [5]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann') 

True

In [6]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[9610, 17272]

In [7]:
shop_data = pd.read_csv("ProductsDataset.csv")


shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [8]:
shop_data.shape

(35548, 8)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [12]:
!ls

ft_model				      index_speaker.pkl    shop.ann
ft_model.trainables.vectors_ngrams_lockf.npy  lesson_10.ipynb	   speaker.ann
ft_model.wv.vectors_ngrams.npy		      lesson_10.zip
index_shop.pkl				      ProductsDataset.csv


In [10]:
with open('index_speaker.pkl', 'rb') as f:
    index_map = pickle.load(f)

In [11]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))

In [12]:
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]

In [13]:
positive_texts = [" ".join(val) for val in shop_data['text'].values]

In [15]:
negative_texts[3]

'приходить вц получение предупреждать утеря чек заполнять заявление сотрудник вц право вскрыть конверт паспорт установить личность сверить дать российский паспорт брать паспорт рф телефон смс готовность присылать паспорт рф штамп номер загран сверять'

In [14]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text'].values]

In [15]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [17]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.979838421391292

In [21]:
import pickle

In [25]:
# assert 
import pickle

with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [20]:
np.mean(tfidf_vect.idf_)

10.91141463828933

In [21]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'готовиться': 8.987703239613996,
 'выпуск': 6.92150692468418,
 'номер': 10.833529930112325,
 'дима': 11.23899503822049,
 'знать': 11.23899503822049,
 'попасть': 11.23899503822049,
 'номинация': 11.23899503822049,
 'представлять': 11.23899503822049,
 'тяжело': 11.23899503822049,
 'расслабляться': 11.23899503822049,
 'станцевать': 11.23899503822049,
 'достойно': 11.23899503822049,
 'заметить': 11.23899503822049,
 'свой': 11.23899503822049,
 'задача': 11.23899503822049,
 'справиться': 11.23899503822049,
 'сто': 11.23899503822049,
 'танец': 11.23899503822049,
 'вишня': 11.23899503822049,
 'дмитрий': 11.23899503822049,
 'юдин': 11.23899503822049,
 'поставить': 11.23899503822049,
 'замечательный': 11.23899503822049,
 'хореограф': 11.23899503822049,
 'саша': 11.23899503822049,
 'могилёв': 11.23899503822049,
 'лариса': 11.23899503822049,
 'полунин': 11.23899503822049,
 'работать': 11.23899503822049,
 'вместе': 11.23899503822049,
 'любовь': 11.23899503822049,
 'светлый': 11.23899503822049,
 'ч

In [35]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

<ipython-input-35-5d70c0b49d4a>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(shop_data))):


True

In [22]:
ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann') 

midf = np.mean(tfidf_vect.idf_)

In [36]:
assert True
import pickle

with open("index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [23]:
'бегать' in modelFT.wv

True

In [25]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    return vector_ft / n_ft

In [26]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 5, include_distances=True)

([394, 906, 1207, 1216, 1271],
 [1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608])

In [31]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [32]:
updater = Updater("1435831861:AAG3gLyDHH3rno8jyce60z1D3kZ4Ih8tU2o", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher

In [33]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

def textMessage(update: Update, context: CallbackContext):
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            update.message.reply_text("title: {} image: {}".format(title, image))
            # bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.35:
        print(distances[0])
        update.message.reply_text("Моя твоя не понимать")
        return
    update.message.reply_text(index_map[ft_index_val[0]])

In [ ]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

# Start the Bot
updater.start_polling()
updater.idle()

<ipython-input-25-eda7a282e6ed>:8: RuntimeWarning: invalid value encountered in true_divide
  return vector_ft / n_ft


1.4142135381698608
